In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_cbohealth_rkhscbo as file

In [2]:
# Args setup
ntrial = 50
n = 100
ntest = 100

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                          seed,
                          n,ntest
                         )
        futures += [f]

In [5]:
# Check on futures
futures

[<Future: pending, key: main-8c781db2d07e0f2c39decbaaca152323>,
 <Future: pending, key: main-3cd08f184df0c7e489af05c58299408a>,
 <Future: pending, key: main-c9fb00f833b5c404a4baf71c2837dbb4>,
 <Future: pending, key: main-b15303be12c66b3b514bd8ce17385903>,
 <Future: pending, key: main-299f8bd1b673256c2b73f2a1c85a079f>,
 <Future: pending, key: main-d4d3001f8c7aa296f78ee9dea44df494>,
 <Future: pending, key: main-928cfeb3ea9dc72b73cf4508ec3ca4c1>,
 <Future: pending, key: main-25bc45c3c0937c4f347f2ce5a058c0ea>,
 <Future: pending, key: main-3789d8b8c0a88b38cce0f70089ce8f7c>,
 <Future: pending, key: main-022b0d86403ed3f6f7a9e06563fbc87e>,
 <Future: pending, key: main-ceac04997cca59ea9240cd25b1fe5f3a>,
 <Future: pending, key: main-9170ce5a2995a6c3c1e6aaab280acc0d>,
 <Future: pending, key: main-0708d386504a336673349542b24c3cf6>,
 <Future: pending, key: main-e0a0c7d2755657ea6a47cea8144fe69d>,
 <Future: pending, key: main-21ff5f13ea67396f766f82df4947aac3>,
 <Future: pending, key: main-816a77b2e27

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "cbohealth_rkhscbo_ntrial={0}_n={1}.pt".format(ntrial,n))